In [ ]:
!pip install pykalman

In [279]:
from pykalman import KalmanFilter
import numpy as np
import pandas as pd
import plotly.graph_objects as plg
import matplotlib.pyplot as plt
from pykalman import KalmanFilter 
from sklearn.metrics import accuracy_score, recall_score, precision_score, roc_auc_score 

Dynamic Linear Models (DLMs) are state-space models in which unobservable states of a system evolve over time. The temporal evolution of the states is represented through a dynamic model, also known as the system equation. Although the states are not directly observed (hence called unobservable), it is possible to observe a function of the states through an observational model or observational equation. DLMs are also referred to as Kalman filters in the engineering literature. The unobservable states a given by:
$$
\boldsymbol{\theta}_{t} = \boldsymbol{G}_{t}\boldsymbol{\theta}_{t-1} + \omega_{t}
$$

where $\boldsymbol{G}_{t}$ is called the evolution matrix and $ω_t$ ∈ $R^n$ represents the stochastic evolution of the system, also known as evolution noise or innovation. Dynamic linear models are Markovian, meaning that the probability associated with a state at time t, given the system's state at time t-1, is independent of the past trajectory. 

It is assumed that: $ω_t ∼ N (0,W_t)$. THus the noise comes from a normal multivariate distribution with covariance $W_t$. Given $\boldsymbol{\theta}_{t}$.Thus we can define the observed state $y_{t}$ by:

$$
    y_t = \boldsymbol{F}_t\boldsymbol{\theta}_{t} + v_t
$$

In which $F_t$ is referred to as the regression matrix and ν$-t$ ∈ R represents the observation errors. It is also assumed that $ν_t$ follows a normal .distribution.



For this toy exemple, we'll use as a dataset the [yahoo dataset](https://www.tumblr.com/yahooresearch/114590420346/a-benchmark-dataset-for-time-series-anomaly), which is a dataset specifically created for anomaly detection tasks. It is labeled, meaning that it contains examples of both normal and anomalous data points, and it is designed to be used for training and evaluating anomaly detection algorithms and models. And to implement the DLM we'll use the [pykalman package](https://pykalman.github.io/). 

In [ ]:
X = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/A1Benchmark/real_13.csv")

anomaly = X[X['is_anomaly'] ==1]
y = X.pop("is_anomaly")


fig = plg.Figure()

# Add traces for each column
fig.add_trace(plg.Scatter(x=X['timestamp'], y=X["value"], name="value"))
fig.add_trace(plg.Scatter(x=anomaly['timestamp'], y=anomaly['value'], mode='markers', marker=dict(color='red', size=10), name='anomaly'))

# Add x-axis and y-axis labels
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Value')

fig.show()

In [ ]:
def detect_anomalies(data, parameters, evolution_matrix, Cov_V, Cov_W, Vetor_reg):
    # Initialize lists to store differences and predicted values
    difs = []
    predição = []
    
    # Initialize KalmanFilter with given parameters
    kf = KalmanFilter(initial_state_mean = parameters,
                      initial_state_covariance=Cov_W,
                      transition_matrices=evolution_matrix,
                      observation_matrices=Vetor_reg,
                      observation_covariance=Cov_V)
    
    # Iterate through the data
    for i in range(len(data)):
        medida = data[i] # Get the current measurement
        
        # Update KalmanFilter with the current measurement
        predicted_state_mean, predicted_state_covariance = kf.filter_update(kf.initial_state_mean,
                                                                              kf.initial_state_covariance, observation=medida)
        
        # Calculate the difference between the current measurement and the predicted state mean
        dif = abs(data[i] - predicted_state_mean[0])
        
        # Append the difference and predicted state mean to the lists
        difs.append(dif)
        predição.append(predicted_state_mean[0])
        
        # Update KalmanFilter's initial state mean and covariance for the next iteration
        kf.initial_state_mean = predicted_state_mean
        kf.initial_state_covariance = predicted_state_covariance
        
    # Calculate the mean and standard deviation of the differences
    mean_difs = np.mean(difs)
    std_difs = np.std(difs)
    
    # Find anomalies by comparing differences to mean + 3 * standard deviation
    anomalies = []
    for i in range(len(difs)):
        if difs[i] > mean_difs + 3 * std_difs:
            anomalies.append(i)
    
    # Return the list of anomalies and the list of predicted state means
    return anomalies, predição

In [276]:
tetha = np.array([0,0]) # unobservable states
G = np.array([[1,1], # evolution_matrix
              [0,1]])

Cov_W = 1*np.eye(2) # covariance matrix of the unobservable states
F = [[1,0]] # regression vector
Cov_V = 1 # covariance of the observable state

In [277]:
# Detect anomalies in X based on "value" column
outlier, Y = detect_anomalies(X["value"], tetha, G, Cov_V, Cov_W, F)

# Select outliers from X
outliers = X.loc[outlier]

# Create an array to store anomaly labels
y_hat = np.zeros((len(y),))

# Set anomaly labels to 1 for identified outliers
y_hat[outlier] = 1

In [278]:
fig = plg.Figure()

# Add traces for each column
fig.add_trace(plg.Scatter(x=X['timestamp'], y=X["value"], name="True Value"))
fig.add_trace(plg.Scatter(x=X['timestamp'], y=Y, name="Prevision"))
fig.add_trace(plg.Scatter(x=anomaly['timestamp'], y=anomaly['value'], mode='markers', marker=dict(color='red', size=10), name='True Anomalyl'))
fig.add_trace(plg.Scatter(x=outliers['timestamp'], y=outliers['value'], mode='markers', marker=dict(color='green', size=10), name='Predict Anomaly'))
# Add x-axis and y-axis labels
fig.update_xaxes(title_text='Date')
fig.update_yaxes(title_text='Value')
print("Recall:", recall_score(y,y_hat))
print("Precision:", precision_score(y,y_hat))
print("Area under ROC:", roc_auc_score(y,y_hat))
fig.show()

Recall: 0.75
Precision: 0.6923076923076923
Area under ROC: 0.8735984583041346
